# Scrape Yelp for Restaurant Review Data in Top 25 Counties Affected by Coronavirus

Using the Yelp API and scraping, collect review data for up to 200 restaurants per county (up to 100 reviews per restaurant) for the top 25 counties with the most coronavirus-related deaths as of April 15, 2020.

This program is ***extremely slow and breaks often*** due to Yelp security - you need to have Yelp open in another window, refreshing every ~1 minute, and be available to answer captcha questions as needed, or else the scraping will return no data. Below, you can see where I've manually stopped the program and run it multiple times to fill in gaps as a result. 

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import urllib
from bs4 import BeautifulSoup
import os
from datetime import date, datetime
from tqdm import tqdm_notebook as tqdm
import time

## Identify top 25 counties with most coronavirus-related deaths
Data collected from the New York Times, available through [Kaggle](https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset) - accurate as of 4/15/2020.


In [2]:
covid_county_data = pd.read_csv('us-counties.csv')
covid_county_data['date'] = pd.to_datetime(covid_county_data['date']).dt.normalize()
#weekly_covid_data = covid_county_data.set_index('date').groupby(['state', 'county']).resample('1W').sum().reset_index().sort_values(['deaths', 'cases'], ascending=False)
#overall_county_data = covid_county_data.groupby(['state', 'county']).sum().reset_index().sort_values(['deaths', 'cases'], ascending=False)
#overall_county_data
newest_county_data = covid_county_data[covid_county_data['date']==covid_county_data['date'].max()].sort_values(['deaths', 'cases'], ascending=False)
top_affected_counties = newest_county_data[['county', 'state']].head(25)
top_affected_counties['county_state'] = top_affected_counties['county'] + str(', ') + top_affected_counties['state']
top_affected_counties = top_affected_counties['county_state'].to_list()
top_affected_counties

['New York City, New York',
 'Nassau, New York',
 'Wayne, Michigan',
 'Westchester, New York',
 'Suffolk, New York',
 'Cook, Illinois',
 'Bergen, New Jersey',
 'Essex, New Jersey',
 'Los Angeles, California',
 'Oakland, Michigan',
 'Fairfield, Connecticut',
 'Macomb, Michigan',
 'King, Washington',
 'Hudson, New Jersey',
 'Orleans, Louisiana',
 'Union, New Jersey',
 'Middlesex, New Jersey',
 'Jefferson, Louisiana',
 'Philadelphia, Pennsylvania',
 'Middlesex, Massachusetts',
 'Hartford, Connecticut',
 'Rockland, New York',
 'Morris, New Jersey',
 'New Haven, Connecticut',
 'Passaic, New Jersey']

In [3]:
yelp_api = os.environ['YELP_API']
headers = {'Authorization': 'Bearer %s' % yelp_api}
url = 'https://api.yelp.com/v3/businesses/search'

business_df = pd.DataFrame()
for c in top_affected_counties:
    # generate up to 200 restaurants per location
    # sort is based on YELP sort
    # only restaurants
    # about 10 miles
    offsets = [0, 50, 100, 150]
    for o in offsets:
        params = {'term':'restaurants', 'location':str(c), 'limit':50, 'radius':16095, 'offset':o}
        req = requests.get(url, params=params, headers=headers)
        parsed = json.loads(req.text)
        businesses = parsed["businesses"]

        # dict for business details
        business_reviews = {}

        for business in businesses:
            business_reviews[business['id']] = business
            business['city'] = c
        for x in business_reviews:
            business_reviews[x].pop('id')
            business_reviews[x].pop('image_url')
            business_reviews[x].pop('url')
            business_reviews[x].pop('phone')    
            business_reviews[x].pop('distance')
            business_reviews[x].pop('display_phone')

        business_df = business_df.append(pd.DataFrame.from_dict(business_reviews, orient='index'))


/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
business_df.index.nunique()

4396

In [5]:
business_df.tail()

,alias,categories,city,coordinates,is_closed,location,name,price,rating,review_count,transactions
srbKFMHjYqVpHJbmwHwl-A,chapos-latin-cuisine-woodland-park,"[{'alias': 'puertorican', 'title': 'Puerto Ric...","Passaic, New Jersey","{'latitude': 40.88985, 'longitude': -74.21497}",False,"{'address1': '71 Lackawanna Ave', 'address2': ...",Chapo's Latin Cuisine,NaN,5.0,7,[delivery]
vHUMukJ0eA4y4i3DMVcj8w,the-crosby-montclair,"[{'alias': 'gastropubs', 'title': 'Gastropubs'}]","Passaic, New Jersey","{'latitude': 40.8135647, 'longitude': -74.2152...",False,"{'address1': '193 Glenridge Ave', 'address2': ...",The Crosby,$$,3.0,517,[]
w8RvA4t6_Mt3WxyEjcslew,punto-fijo-restaurant-ii-passaic,"[{'alias': 'latin', 'title': 'Latin American'}...","Passaic, New Jersey","{'latitude': 40.86132, 'longitude': -74.12816}",False,"{'address1': '30 Howe Ave', 'address2': '', 'a...",Punto Fijo Restaurant II,NaN,5.0,1,[delivery]
wvV7-gOilUd0MfZKVN0o9Q,laurel-and-sage-montclair,"[{'alias': 'newamerican', 'title': 'American (...","Passaic, New Jersey","{'latitude': 40.8136520385742, 'longitude': -7...",False,"{'address1': '33 Walnut St', 'address2': '', '...",Laurel & Sage,$$$,4.0,143,[delivery]
yQ8CA83K5MCK-Zbq5zo_Ng,belmont-tavern-belleville,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","Passaic, New Jersey","{'latitude': 40.7750255, 'longitude': -74.1863...",False,"{'address1': '12 Bloomfield Ave', 'address2': ...",Belmont Tavern,$$,4.0,171,[delivery]


In [6]:
## use scraping to get more reviews
## print the first 100 reviews                
def scrape_reviews(input_df):
    for c in input_df['city'].unique():
        df = input_df[input_df['city']==c]
        print('Scraping Reviews for {} ({} restaurants)'.format(c, len(df)))        
        for restaurant_id in tqdm(df.index):
            all_reviews = {}
            restaurant_reviews[restaurant_id] = all_reviews
            url_r = "https://www.yelp.com/biz/" + restaurant_id

            start = 0
            num_pages = 5
            end = 20 * num_pages

            authors = []
            publish_date = []
            reviews = []

            counter = 0
            while (start < end):
                url = url_r + '?start=' + str(start)
                start += 20

                try:
                    with urllib.request.urlopen(url) as url:
                        page = url.read()
                        soup = BeautifulSoup(page, "html.parser") 

                except urllib.error.HTTPError:
                    time.sleep(0.1)
                    continue

                for s in soup.find_all('div',{'itemprop':'review'}):
                    counter += 1
                    review = {}
                    all_reviews[counter] = review
                    for author in s.find_all('meta', {'itemprop':'author'}):
                        author = str(author)[15:-21]
                        review['author'] = author
                    for date in s.find_all('meta', {'itemprop':'datePublished'}):
                        publish_date = str(date)[15:-28]
                        review['publish_date'] = publish_date
                    for descrip in s.find_all('p', {'itemprop':'description'}):
                        descrip = str(descrip)[26:-12]
                        review['description'] = descrip
                    for score in s.find_all('meta', {'itemprop':'ratingValue'}):
                        score = float(str(score)[15:-26])
                        review['score'] = score
                    review['city'] = c

In [8]:
restaurant_reviews = {}
scrape_reviews(business_df)

Scraping Reviews for New York City, New York (200 restaurants)



Scraping Reviews for Nassau, New York (55 restaurants)



Scraping Reviews for Wayne, Michigan (200 restaurants)



Scraping Reviews for Westchester, New York (200 restaurants)



Scraping Reviews for Suffolk, New York (200 restaurants)


...


Scraping Reviews for Hartford, Connecticut (200 restaurants)



Scraping Reviews for Rockland, New York (200 restaurants)



Scraping Reviews for Morris, New Jersey (200 restaurants)



Scraping Reviews for New Haven, Connecticut (200 restaurants)



Scraping Reviews for Passaic, New Jersey (200 restaurants)


In [70]:
# parse nested dictionary into a dataframe
df1 = pd.DataFrame()
for restaurant in restaurant_reviews:
    restaurant_df = pd.DataFrame.from_dict(restaurant_reviews[restaurant], orient='index')
    restaurant_df['id'] = restaurant
    restaurant_df = restaurant_df.reset_index().set_index('id').rename(columns={'index':'review_number'})
    df1 = df1.append(restaurant_df)
df1.head()

,author,city,description,publish_date,review_number,score
id,,,,,,
3gqKLANaXNsHZnPDjKrcTg,Evelyn W.,"New York City, New York","100% Asian Fusion. \n\nDecor is cute, service ...",2020-03-07,1,5.0
3gqKLANaXNsHZnPDjKrcTg,Diana X.,"New York City, New York","It's cool to see a more ""authentic"" take on As...",2020-03-29,2,4.0
3gqKLANaXNsHZnPDjKrcTg,Tina F.,"New York City, New York",I don't know how to start. This place gets rea...,2020-03-01,3,3.0
3gqKLANaXNsHZnPDjKrcTg,Chey O.,"New York City, New York","Yeah, the place really is as good as people sa...",2020-02-21,4,5.0
3gqKLANaXNsHZnPDjKrcTg,Jin C.,"New York City, New York",I came here a little before 7p on a Thursday n...,2020-02-02,5,4.0


In [71]:
review_data = business_df.merge(df1, how='left', left_index=True, right_index=True)
review_data.head()

,alias,categories,city_x,coordinates,is_closed,location,name,price,rating,review_count,transactions,author,city_y,description,publish_date,review_number,score
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-1B9pP_CrRBJYPICE5WbRA,spice-28-philadelphia,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...","Philadelphia, Pennsylvania","{'latitude': 39.950352, 'longitude': -75.1615829}",False,"{'address1': '1228 Chestnut St', 'address2': '...",Spice 28,$$,4.0,795,"[restaurant_reservation, delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1OKn2TRxsoZ1fZClMnnog,j-gilberts-wood-fired-steaks-and-seafood-glast...,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...","Hartford, Connecticut","{'latitude': 41.721484, 'longitude': -72.622915}",False,"{'address1': '185 Glastonbury Blvd', 'address2...",J. Gilbert's Wood-Fired Steaks & Seafood Glast...,$$$,4.5,589,"[delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1XSzguS6XLN-V6MVZMg2A,restaurant-rebirth-new-orleans,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...","Jefferson, Louisiana","{'latitude': 29.943528, 'longitude': -90.0654044}",False,"{'address1': '857 Fulton St', 'address2': None...",Restaurant Rebirth,$$$,4.5,463,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-2oop6EYGWRY9avJWKbqYQ,four-brothers-pizza-and-restaurant-valatie,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","Nassau, New York","{'latitude': 42.428024, 'longitude': -73.687987}",False,"{'address1': 'RR 9', 'address2': '', 'address3...",Four Brothers Pizza & Restaurant,$$,3.0,41,[],John C.,"Nassau, New York",I don't know why they only average 3 stars and...,2020-02-01,1,4.0


In [80]:
# clearly some duplicates because of overlapping counties
len(review_data[review_data.duplicated(subset='description')])

3073

In [81]:
len(review_data)

154197

In [82]:
# save csv with what we have
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
review_data.to_csv('review_data_{}.csv'.format(now))

In [83]:
# rescrape businesses with no review data
rescrape = review_data[review_data['publish_date'].isna()].index.to_list()
rescrape_df = business_df[business_df.index.isin(rescrape)]
len(business_df), len(rescrape_df)

(4639, 2501)

In [39]:
## use scraping to get more reviews
## print the first 100 reviews                
def rescrape_reviews(input_df, output_dict):
    for c in input_df['city'].unique():
        df = input_df[input_df['city']==c]
        print('Scraping Reviews for {} ({} restaurants)'.format(c, len(df)))        
        for restaurant_id in tqdm(df.index):
            all_reviews = {}
            output_dict[restaurant_id] = all_reviews
            url_r = "https://www.yelp.com/biz/" + restaurant_id

            start = 0
            num_pages = 5
            end = 20 * num_pages

            authors = []
            publish_date = []
            reviews = []

            counter = 0
            while (start < end):
                url = url_r + '?start=' + str(start)
                start += 20

                try:
                    with urllib.request.urlopen(url) as url:
                        page = url.read()
                        soup = BeautifulSoup(page, "html.parser") 

                except urllib.error.HTTPError:
                    time.sleep(0.1)
                    continue

                for s in soup.find_all('div',{'itemprop':'review'}):
                    counter += 1
                    review = {}
                    all_reviews[counter] = review
                    for author in s.find_all('meta', {'itemprop':'author'}):
                        author = str(author)[15:-21]
                        review['author'] = author
                    for date in s.find_all('meta', {'itemprop':'datePublished'}):
                        publish_date = str(date)[15:-28]
                        review['publish_date'] = publish_date
                    for descrip in s.find_all('p', {'itemprop':'description'}):
                        descrip = str(descrip)[26:-12]
                        review['description'] = descrip
                    for score in s.find_all('meta', {'itemprop':'ratingValue'}):
                        score = float(str(score)[15:-26])
                        review['score'] = score
                    review['city'] = c

In [44]:
rescraped_reviews = {}
rescrape_reviews(rescrape_df, rescraped_reviews)

Scraping Reviews for New York City, New York (65 restaurants)


Scraping Reviews for Suffolk, New York (1 restaurants)


Scraping Reviews for Bergen, New Jersey (65 restaurants)


Scraping Reviews for Essex, New Jersey (81 restaurants)


Scraping Reviews for Fairfield, Connecticut (1 restaurants)


Scraping Reviews for Hudson, New Jersey (176 restaurants)


Scraping Reviews for Orleans, Louisiana (112 restaurants)


Scraping Reviews for Union, New Jersey (200 restaurants)


Scraping Reviews for Middlesex, New Jersey (200 restaurants)


KeyboardInterrupt: 

In [99]:
# had to interrupt - was stuck for >1 h w/ no progress after yelp wanted to verify not a robot

In [46]:
# parse nested dictionary into a dataframe
df2 = pd.DataFrame()
for restaurant in rescraped_reviews:
    restaurant_df = pd.DataFrame.from_dict(rescraped_reviews[restaurant], orient='index')
    restaurant_df['id'] = restaurant
    restaurant_df = restaurant_df.reset_index().set_index('id').rename(columns={'index':'review_number'})
    df2 = df2.append(restaurant_df)
    
df2.head()

42297

In [91]:
# combine reviews scraped so far
current_reviews = df.append(df2)

# no duplicates
current_reviews[current_reviews.duplicated()]

,author,city,description,publish_date,review_number,score
id,,,,,,


In [94]:
review_data = business_df.merge(current_reviews, how='left', left_index=True, right_index=True)
review_data.head()

,alias,categories,city_x,coordinates,is_closed,location,name,price,rating,review_count,transactions,author,city_y,description,publish_date,review_number,score
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-1B9pP_CrRBJYPICE5WbRA,spice-28-philadelphia,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...","Philadelphia, Pennsylvania","{'latitude': 39.950352, 'longitude': -75.1615829}",False,"{'address1': '1228 Chestnut St', 'address2': '...",Spice 28,$$,4.0,795,"[restaurant_reservation, delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1OKn2TRxsoZ1fZClMnnog,j-gilberts-wood-fired-steaks-and-seafood-glast...,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...","Hartford, Connecticut","{'latitude': 41.721484, 'longitude': -72.622915}",False,"{'address1': '185 Glastonbury Blvd', 'address2...",J. Gilbert's Wood-Fired Steaks & Seafood Glast...,$$$,4.5,589,"[delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1XSzguS6XLN-V6MVZMg2A,restaurant-rebirth-new-orleans,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...","Jefferson, Louisiana","{'latitude': 29.943528, 'longitude': -90.0654044}",False,"{'address1': '857 Fulton St', 'address2': None...",Restaurant Rebirth,$$$,4.5,463,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-2oop6EYGWRY9avJWKbqYQ,four-brothers-pizza-and-restaurant-valatie,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","Nassau, New York","{'latitude': 42.428024, 'longitude': -73.687987}",False,"{'address1': 'RR 9', 'address2': '', 'address3...",Four Brothers Pizza & Restaurant,$$,3.0,41,[],John C.,"Nassau, New York",I don't know why they only average 3 stars and...,2020-02-01,1,4.0


In [95]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
review_data.to_csv('review_data_{}.csv'.format(now))

In [104]:
# rescrape businesses with no review data
rescrape = review_data[review_data['publish_date'].isna()].index.to_list()
rescrape_df = business_df[business_df.index.isin(rescrape)]
len(business_df), len(rescrape_df)

(4639, 1689)

In [105]:
rescraped_reviews_2 = {}
rescrape_reviews(rescrape_df, rescraped_reviews_2)

Scraping Reviews for New York City, New York (10 restaurants)


Scraping Reviews for Suffolk, New York (1 restaurants)


Scraping Reviews for Bergen, New Jersey (7 restaurants)


Scraping Reviews for Fairfield, Connecticut (1 restaurants)


Scraping Reviews for Hudson, New Jersey (25 restaurants)


Scraping Reviews for Middlesex, New Jersey (146 restaurants)


KeyboardInterrupt: 

In [ ]:
# stopped script to check something out

In [106]:
# parse nested dictionary into a dataframe
df3 = pd.DataFrame()
for restaurant in rescraped_reviews_2:
    restaurant_df = pd.DataFrame.from_dict(rescraped_reviews_2[restaurant], orient='index')
    restaurant_df['id'] = restaurant
    restaurant_df = restaurant_df.reset_index().set_index('id').rename(columns={'index':'review_number'})
    df3 = df3.append(restaurant_df)
    
df3.head()

,author,city,description,publish_date,review_number,score
id,,,,,,
0CjK3esfpFcxIopebzjFxA,Sarah M.,"Hudson, New Jersey",I love the new location. It is very hip! Somet...,2020-04-14,1,5.0
0CjK3esfpFcxIopebzjFxA,Al S.,"Hudson, New Jersey",They treat their customers like crap. The wai...,2020-04-17,2,1.0
0CjK3esfpFcxIopebzjFxA,Christa P.,"Hudson, New Jersey",Came here on monday afternoon where the restau...,2020-04-09,3,1.0
0CjK3esfpFcxIopebzjFxA,Tyler T.,"Hudson, New Jersey","Honestly, pretty average Xiao Long Bao, I'm no...",2020-04-04,4,3.0
0CjK3esfpFcxIopebzjFxA,Cynthia S.,"Hudson, New Jersey",I Never wanted to try Joe's Shanghai based on ...,2020-02-20,5,2.0


In [107]:
# combine reviews scraped so far
current_reviews_2 = current_reviews.append(df3)

# no duplicates
current_reviews_2[current_reviews_2.duplicated()]

,author,city,description,publish_date,review_number,score
id,,,,,,


In [109]:
review_data = business_df.merge(current_reviews_2, how='left', left_index=True, right_index=True)
review_data.head()

,alias,categories,city_x,coordinates,is_closed,location,name,price,rating,review_count,transactions,author,city_y,description,publish_date,review_number,score
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-1B9pP_CrRBJYPICE5WbRA,spice-28-philadelphia,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...","Philadelphia, Pennsylvania","{'latitude': 39.950352, 'longitude': -75.1615829}",False,"{'address1': '1228 Chestnut St', 'address2': '...",Spice 28,$$,4.0,795,"[restaurant_reservation, delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1OKn2TRxsoZ1fZClMnnog,j-gilberts-wood-fired-steaks-and-seafood-glast...,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...","Hartford, Connecticut","{'latitude': 41.721484, 'longitude': -72.622915}",False,"{'address1': '185 Glastonbury Blvd', 'address2...",J. Gilbert's Wood-Fired Steaks & Seafood Glast...,$$$,4.5,589,"[delivery, pickup]",NaN,NaN,NaN,NaN,NaN,NaN
-1XSzguS6XLN-V6MVZMg2A,restaurant-rebirth-new-orleans,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...","Jefferson, Louisiana","{'latitude': 29.943528, 'longitude': -90.0654044}",False,"{'address1': '857 Fulton St', 'address2': None...",Restaurant Rebirth,$$$,4.5,463,[delivery],NaN,NaN,NaN,NaN,NaN,NaN
-2oop6EYGWRY9avJWKbqYQ,four-brothers-pizza-and-restaurant-valatie,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","Nassau, New York","{'latitude': 42.428024, 'longitude': -73.687987}",False,"{'address1': 'RR 9', 'address2': '', 'address3...",Four Brothers Pizza & Restaurant,$$,3.0,41,[],John C.,"Nassau, New York",I don't know why they only average 3 stars and...,2020-02-01,1,4.0


In [110]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
review_data.to_csv('review_data_{}.csv'.format(now))

In [111]:
# rescrape businesses with no review data
rescrape = review_data[review_data['publish_date'].isna()].index.to_list()
rescrape_df = business_df[business_df.index.isin(rescrape)]
len(business_df), len(rescrape_df)

(4639, 1581)

In [112]:
# go again
rescraped_reviews_3 = {}
rescrape_reviews(rescrape_df, rescraped_reviews_3)

Scraping Reviews for Suffolk, New York (1 restaurants)


Scraping Reviews for Fairfield, Connecticut (1 restaurants)


Scraping Reviews for Middlesex, New Jersey (91 restaurants)


Scraping Reviews for Jefferson, Louisiana (200 restaurants)


Scraping Reviews for Philadelphia, Pennsylvania (200 restaurants)


...

Scraping Reviews for Hartford, Connecticut (200 restaurants)


Scraping Reviews for Rockland, New York (197 restaurants)


Scraping Reviews for Morris, New Jersey (198 restaurants)


Scraping Reviews for New Haven, Connecticut (200 restaurants)


Scraping Reviews for Passaic, New Jersey (93 restaurants)


In [113]:
# parse nested dictionary into a dataframe
df4 = pd.DataFrame()
for restaurant in rescraped_reviews_3:
    restaurant_df = pd.DataFrame.from_dict(rescraped_reviews_3[restaurant], orient='index')
    restaurant_df['id'] = restaurant
    restaurant_df = restaurant_df.reset_index().set_index('id').rename(columns={'index':'review_number'})
    df4 = df4.append(restaurant_df)
    
df4.head()

,author,city,description,publish_date,review_number,score
id,,,,,,
7-5xaHkgw05ZrykGVcFeMA,Merari G.,"Middlesex, New Jersey",Started with cheesesteaks as it came as a reco...,2020-03-02,1,5.0
7-5xaHkgw05ZrykGVcFeMA,Angel R.,"Middlesex, New Jersey",Real good quality pizza and a friendly staff. ...,2019-11-15,2,5.0
7-5xaHkgw05ZrykGVcFeMA,Melissa S.,"Middlesex, New Jersey",We haven't tried the pizza yet (go figure) but...,2018-12-17,3,5.0
7-5xaHkgw05ZrykGVcFeMA,Leah B.,"Middlesex, New Jersey",I literally just threw away an entire eggplant...,2019-05-13,4,1.0
7-5xaHkgw05ZrykGVcFeMA,Jane D.,"Middlesex, New Jersey",This is an updated review since my last visit....,2018-04-30,5,2.0


In [114]:
### combine reviews scraped so far
current_reviews_3 = current_reviews_2.append(df4)

# no duplicates
current_reviews_3[current_reviews_3.duplicated()]

,author,city,description,publish_date,review_number,score
id,,,,,,
PdmsXmprYb_-GdbfjNp7Gg,Harrison E.,"Hartford, Connecticut",Best Italian food in Hartford County! Not an e...,2020-04-05,2,5.0


In [115]:
review_data = business_df.merge(current_reviews_3, how='left', left_index=True, right_index=True)
review_data.head()

,alias,categories,city_x,coordinates,is_closed,location,name,price,rating,review_count,transactions,author,city_y,description,publish_date,review_number,score
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Sara P.,"Middlesex, Massachusetts",I haven't even left yet. But I am thoroughly p...,2020-01-24,1,5.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Matt K.,"Middlesex, Massachusetts",I've been to Welly's a few times and each time...,2019-11-12,2,4.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],AnnMarie H.,"Middlesex, Massachusetts",Welly's is a very popular bar right in downtow...,2019-11-08,3,4.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Christy C.,"Middlesex, Massachusetts",I used to think the idea of fish tacos were no...,2020-04-03,4,5.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Marissa V.,"Middlesex, Massachusetts",Came here with some work colleagues and ordere...,2020-01-03,5,4.0


In [116]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
review_data.to_csv('review_data_{}.csv'.format(now))

In [117]:
len(review_data)

333103

In [118]:
# rescrape businesses with no review data
rescrape = review_data[review_data['publish_date'].isna()].index.to_list()
rescrape_df = business_df[business_df.index.isin(rescrape)]
len(business_df), len(rescrape_df)

(4639, 116)

In [119]:
# go again
rescraped_reviews_4 = {}
rescrape_reviews(rescrape_df, rescraped_reviews_4)

Scraping Reviews for Suffolk, New York (1 restaurants)


Scraping Reviews for Fairfield, Connecticut (1 restaurants)


Scraping Reviews for Middlesex, New Jersey (50 restaurants)


Scraping Reviews for Jefferson, Louisiana (52 restaurants)


Scraping Reviews for Philadelphia, Pennsylvania (6 restaurants)


Scraping Reviews for Hartford, Connecticut (3 restaurants)


Scraping Reviews for Rockland, New York (3 restaurants)


In [120]:
# parse nested dictionary into a dataframe
df5 = pd.DataFrame()
for restaurant in rescraped_reviews_4:
    restaurant_df = pd.DataFrame.from_dict(rescraped_reviews_4[restaurant], orient='index')
    restaurant_df['id'] = restaurant
    restaurant_df = restaurant_df.reset_index().set_index('id').rename(columns={'index':'review_number'})
    df5 = df5.append(restaurant_df)
    
df5.head()

,author,city,description,publish_date,review_number,score
id,,,,,,
-waToVFx_mUO8qwvv8fd4A,Donna R.,"Middlesex, New Jersey","This is definitely a dive style bar, but let m...",2016-03-18,1,4.0
-waToVFx_mUO8qwvv8fd4A,Kirsten K.,"Middlesex, New Jersey",Tim Kerwin's is a fun Irish bar that serves a ...,2019-01-20,2,5.0
-waToVFx_mUO8qwvv8fd4A,Lisa H.,"Middlesex, New Jersey",If your looking for a fun relaxed good time th...,2019-07-09,3,4.0
-waToVFx_mUO8qwvv8fd4A,Mark P.,"Middlesex, New Jersey",Great place to shoot darts and hang out! With ...,2019-01-26,4,5.0
-waToVFx_mUO8qwvv8fd4A,Wendy S.,"Middlesex, New Jersey",Cant wait for the weekends when my favorite ba...,2019-01-26,5,5.0


In [121]:
### combine reviews scraped so far
current_reviews_4 = current_reviews_3.append(df5)

# no duplicates
current_reviews_4[current_reviews_4.duplicated()]

,author,city,description,publish_date,review_number,score
id,,,,,,
PdmsXmprYb_-GdbfjNp7Gg,Harrison E.,"Hartford, Connecticut",Best Italian food in Hartford County! Not an e...,2020-04-05,2,5.0


In [122]:
review_data = business_df.merge(current_reviews_4, how='left', left_index=True, right_index=True)
review_data.head()

,alias,categories,city_x,coordinates,is_closed,location,name,price,rating,review_count,transactions,author,city_y,description,publish_date,review_number,score
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Sara P.,"Middlesex, Massachusetts",I haven't even left yet. But I am thoroughly p...,2020-01-24,1,5.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Matt K.,"Middlesex, Massachusetts",I've been to Welly's a few times and each time...,2019-11-12,2,4.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],AnnMarie H.,"Middlesex, Massachusetts",Welly's is a very popular bar right in downtow...,2019-11-08,3,4.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Christy C.,"Middlesex, Massachusetts",I used to think the idea of fish tacos were no...,2020-04-03,4,5.0
--CprxtcUfzKoz29hAzm5w,wellys-restaurant-marlborough,"[{'alias': 'tradamerican', 'title': 'American ...","Middlesex, Massachusetts","{'latitude': 42.3469, 'longitude': -71.54837}",False,"{'address1': '153 Main St', 'address2': '', 'a...",Welly's Restaurant,$$,4.0,170,[delivery],Marissa V.,"Middlesex, Massachusetts",Came here with some work colleagues and ordere...,2020-01-03,5,4.0


In [123]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
review_data.to_csv('review_data_{}.csv'.format(now))

In [124]:
len(review_data)

342034

In [125]:
# rescrape businesses with no review data
rescrape = review_data[review_data['publish_date'].isna()].index.to_list()
rescrape_df = business_df[business_df.index.isin(rescrape)]
len(business_df), len(rescrape_df)

(4639, 2)

In [127]:
# don't worry about rescraping last 2 businesses
rescrape_df

,alias,categories,city,coordinates,is_closed,location,name,price,rating,review_count,transactions
buZxGdNGUuRl19gVa8GL0Q,jr-el-mariachi-mexican-restaurant-riverhead,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","Suffolk, New York","{'latitude': 40.91392, 'longitude': -72.65677}",False,"{'address1': '191 Flanders Rd', 'address2': No...",JR El Mariachi Mexican Restaurant,NaN,1.0,1,[delivery]
X-GDbySI-heuP_l1xJ2Zpg,the-chapel-margate,"[{'alias': 'british', 'title': 'British'}, {'a...","Fairfield, Connecticut","{'latitude': 51.3788, 'longitude': 1.40153}",False,"{'address1': '44 46 Albion Street', 'address2'...",The Chapel,NaN,5.0,1,[]


In [142]:
# double check that full reviews are present
review_data.loc['--CprxtcUfzKoz29hAzm5w', 'description'].apply(lambda x: print(x))

I haven't even left yet. But I am thoroughly pleased with our dinner choice. First there is a huge hockey party here and they were like we won't sit you next to them if you don't want to but we have a kid and we're like no worries. Paul was attentive and kind. He had great suggestions for us.


 We had the Chicken Carbonara and the Nachos with Chilli. And OMFG it was so good. The chicken was so tender and delightful the Nachos were phenomenal. You'd be surprised how many places mess up nachos, but these guys made them just right. Crispy chips, cheesey, delicious and with fresh tomatoes to boot. My husband and I had to stop ourselves from finishing them. There was no more room in our bellies, but our taste buds demanded more.

I'm stoked for the new location in Hudson it cannot open fast enough!

I have to stop eating. It's so good! I cannot.
I've been to Welly's a few times and each time, the food has been incredible! Most recently I had the veal/eggplant parm combo, which was deliciou

--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
--CprxtcUfzKoz29hAzm5w    None
                          ... 
--CprxtcUfzKoz29hAzm5w    None
--Cprxtc